In [1]:
import sys
root = '/home/featurize/data'
sys.path.append(f'{root}/pikachu/utils')
sys.path.append(f'{root}/pikachu/third')
sys.path.append(f'{root}/pikachu')
sys.path.append(f'{root}/pikachu/projects/ai/naic_seg')

In [2]:
from IPython.display import display
import os, glob
import numpy as np
import cv2
import tensorflow as tf
import pandas as pd
import gezi
from gezi import tqdm
from gezi.metrics.image.semantic_seg import Evaluator
import melt as mt
from gseg.dataset import Dataset
from gseg.loss import get_loss_fn
from gseg.metrics import get_metrics
from gseg import util
from gseg.util import *
gezi.set_pandas()

In [3]:
import mindspore
import mindspore as ms
physical_devices = tf.config.list_physical_devices('GPU') 
tf.config.set_visible_devices(physical_devices, 'GPU')
tf.config.set_soft_device_placement(True)

In [10]:
from mindspore import Model, context
from mindspore.nn import WithLossCell, TrainOneStepCell
from mindspore.nn import Momentum, AdamWeightDecay
device_id = 0
context.set_context(mode=context.PYNATIVE_MODE, device_target="GPU", save_graphs=False, device_id=device_id)
mindspore.set_seed(1)

In [11]:
CLASSES = ['water', 'track_road', 'build', 'track_airport', 'other_park', 'other_playground', 'arable_natural', 'arable_greenhouse',
           'grass_natural', 'grass_greenbelt', 'forest_natural', 'forest_planted', 'bare_natural', 'bare_planted', 'other_other']
NUM_CLASSES = len(CLASSES)

In [12]:
mt.init_flags()
FLAGS = mt.get_flags()
FLAGS.augment_level = 7
FLAGS.cutmix_rate=1. 
FLAGS.cutmix_range = [0.9, 1.1]
FLAGS.batch_parse = False
batch_size = 2
mt.set_global('batch_size', batch_size) # loss fn used / mt.batch_size()
eval_files = gezi.list_files('../input/quarter/tfrecords/train/1/21*')
train_files = gezi.list_files('../input/quarter/tfrecords/train/*/*')
train_files = [x for x in train_files if x not in eval_files]
# print(len(train_files), len(eval_files))
eval_dataset = Dataset('valid').make_batch(batch_size, eval_files)
train_dataset = Dataset('train').make_batch(batch_size, train_files)
train_steps = -(-mt.get_num_records(train_files) // batch_size)
steps = -(-mt.get_num_records(eval_files) // batch_size)
FLAGS.NUM_CLASSES = NUM_CLASSES

In [13]:
import mindspore.nn as nn
import mindspore.ops.operations as F
from mindspore.common.initializer import TruncatedNormal
from mindspore.nn import CentralCrop

class DoubleConv(nn.Cell):

    def __init__(self, in_channels, out_channels, mid_channels=None):
        super().__init__()
        init_value_0 = TruncatedNormal(0.06)
        init_value_1 = TruncatedNormal(0.06)
        if not mid_channels:
            mid_channels = out_channels
        self.double_conv = nn.SequentialCell(
            [nn.Conv2d(in_channels, mid_channels, kernel_size=3, has_bias=True,
                       weight_init=init_value_0, pad_mode="valid"),
             nn.ReLU(),
             nn.Conv2d(mid_channels, out_channels, kernel_size=3, has_bias=True,
                       weight_init=init_value_1, pad_mode="valid"),
             nn.ReLU()]
        )

    def construct(self, x):
        return self.double_conv(x)


class Down(nn.Cell):
    """Downscaling with maxpool then double conv"""

    def __init__(self, in_channels, out_channels):
        super().__init__()

        self.maxpool_conv = nn.SequentialCell(
            [nn.MaxPool2d(kernel_size=2, stride=2),
             DoubleConv(in_channels, out_channels)]
        )

    def construct(self, x):
        return self.maxpool_conv(x)


class Up1(nn.Cell):
    """Upscaling then double conv"""

    def __init__(self, in_channels, out_channels, bilinear=True):
        super().__init__()
        self.concat = F.Concat(axis=1)
        self.factor = 56.0 / 64.0
        self.center_crop = CentralCrop(central_fraction=self.factor)
        self.print_fn = F.Print()
        self.conv = DoubleConv(in_channels, out_channels, in_channels // 2)
        self.up = nn.Conv2dTranspose(in_channels, in_channels // 2, kernel_size=2, stride=2)
        self.relu = nn.ReLU()

    def construct(self, x1, x2):
        x1 = self.up(x1)
        x1 = self.relu(x1)
        x2 = self.center_crop(x2)
        x = self.concat((x1, x2))
        return self.conv(x)


class Up2(nn.Cell):
    """Upscaling then double conv"""

    def __init__(self, in_channels, out_channels, bilinear=True):
        super().__init__()
        self.concat = F.Concat(axis=1)
        self.factor = 104.0 / 136.0
        self.center_crop = CentralCrop(central_fraction=self.factor)
        self.conv = DoubleConv(in_channels, out_channels, in_channels // 2)
        self.up = nn.Conv2dTranspose(in_channels, in_channels // 2, kernel_size=2, stride=2)
        self.relu = nn.ReLU()

    def construct(self, x1, x2):
        x1 = self.up(x1)
        x1 = self.relu(x1)
        x2 = self.center_crop(x2)
        x = self.concat((x1, x2))
        return self.conv(x)


class Up3(nn.Cell):
    """Upscaling then double conv"""

    def __init__(self, in_channels, out_channels, bilinear=True):
        super().__init__()
        self.concat = F.Concat(axis=1)
        self.factor = 200 / 280
        self.center_crop = CentralCrop(central_fraction=self.factor)
        self.print_fn = F.Print()
        self.conv = DoubleConv(in_channels, out_channels, in_channels // 2)
        self.up = nn.Conv2dTranspose(in_channels, in_channels // 2, kernel_size=2, stride=2)
        self.relu = nn.ReLU()

    def construct(self, x1, x2):
        x1 = self.up(x1)
        x1 = self.relu(x1)
        x2 = self.center_crop(x2)
        x = self.concat((x1, x2))
        return self.conv(x)


class Up4(nn.Cell):
    """Upscaling then double conv"""

    def __init__(self, in_channels, out_channels, bilinear=True):
        super().__init__()
        self.concat = F.Concat(axis=1)
        self.factor = 392 / 568
        self.center_crop = CentralCrop(central_fraction=self.factor)
        self.conv = DoubleConv(in_channels, out_channels, in_channels // 2)
        self.up = nn.Conv2dTranspose(in_channels, in_channels // 2, kernel_size=2, stride=2)
        self.relu = nn.ReLU()

    def construct(self, x1, x2):
        x1 = self.up(x1)
        x1 = self.relu(x1)
        x2 = self.center_crop(x2)
        x = self.concat((x1, x2))
        return self.conv(x)


class OutConv(nn.Cell):
    def __init__(self, in_channels, out_channels):
        super(OutConv, self).__init__()
        init_value = TruncatedNormal(0.06)
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=1, has_bias=True, weight_init=init_value)

    def construct(self, x):
        x = self.conv(x)
        return x
      
class UNet(nn.Cell):
    def __init__(self, n_channels, n_classes):
        super(UNet, self).__init__()
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.inc = DoubleConv(n_channels, 64)
        self.down1 = Down(64, 128)
        self.down2 = Down(128, 256)
        self.down3 = Down(256, 512)
        self.down4 = Down(512, 1024)
        self.up1 = Up1(1024, 512)
        self.up2 = Up2(512, 256)
        self.up3 = Up3(256, 128)
        self.up4 = Up4(128, 64)
        self.outc = OutConv(64, n_classes)

    def construct(self, x):

        x1 = self.inc(x)
        x2 = self.down1(x1)
        x3 = self.down2(x2)
        x4 = self.down3(x3)
        x5 = self.down4(x4)
        x = self.up1(x5, x4)
        x = self.up2(x, x3)
        x = self.up3(x, x2)
        x = self.up4(x, x1)
        logits = self.outc(x)
        logits = tf.transpose(logits, [0,2,3,1])
        return logits

In [14]:
net = UNet(3, NUM_CLASSES)
loss = nn.SoftmaxCrossEntropyWithLogits()
optimizer = Momentum(params=net.trainable_params(), learning_rate=0.1, momentum=0.9)
net_with_loss = WithLossCell(net, loss)
train_network = TrainOneStepCell(net_with_loss, optimizer)
train_network.set_train()
pass

In [15]:
for x, y in train_dataset:
  x['image'] = tf.image.resize(x['image'], [572, 572])
  y = tf.image.resize(y, [388, 388], method='nearest')
  y = tf.squeeze(y, -1)
  y = tf.one_hot(tf.cast(y, tf.int32), NUM_CLASSES)
  x['image'] = tf.transpose(x['image'], [0, 3, 1, 2])

  for key in x:
    x[key] = ms.Tensor(x[key].numpy())
  y = ms.Tensor(y.numpy())
  print(x['image'].shape, y.shape)
#   print(x['image'], y)
#   print(model(x['image']))
  # Run training 
  output = train_network(x['image'], y)
  break

(2, 3, 572, 572) (2, 388, 388, 15)


ValueError: Cannot infer dtype of a Tensor object, as iter(<object>).next() failed: TypeError: When tensor is indexed by an integer,the dimension of the tensor cannot be 0.
Traceback (most recent call last):

  File "/environment/python/versions/3.7.4/lib/python3.7/site-packages/mindspore/common/tensor.py", line 165, in __getitem__
    out = tensor_operator_registry.get('__getitem__')(self, index)

  File "/environment/python/versions/3.7.4/lib/python3.7/site-packages/mindspore/ops/composite/multitype_ops/_compile_utils.py", line 174, in _tensor_getitem
    return _tensor_index_by_integer(self, index)

  File "/environment/python/versions/3.7.4/lib/python3.7/site-packages/mindspore/ops/composite/multitype_ops/_compile_utils.py", line 220, in _tensor_index_by_integer
    return const_utils.raise_type_error("When tensor is indexed by an integer,"

  File "/environment/python/versions/3.7.4/lib/python3.7/site-packages/mindspore/ops/primitive.py", line 159, in __call__
    return _run_op(s